In [5]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from scipy.special import digamma
import csv

import pandas as pd
from sklearn.model_selection import train_test_split

# Configuración
NUM_FACTORS = 5
ALPHA = 0.8
BETA = 5
R = 4
MIN_RATING = 0
MAX_RATING = 10
NUM_ITERATIONS = 50

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')


In [2]:
!pip install scikit-surprise numpy==1.26.4

In [3]:
!pip list | grep numpy

numpy                              1.26.4


In [7]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.trainset import Trainset
import csv

# https://surpriselib.com/

# Cargar datos de entrenamiento
df_train = pd.read_csv("train.csv")
reader = Reader(rating_scale=(0, 10))  # Asegúrate que coincide con tu escala
data = Dataset.load_from_df(df_train[['user', 'item', 'rating']], reader)
trainset: Trainset = data.build_full_trainset()

# Entrenar modelo SVD
algo = SVD(n_factors=50, n_epochs=20, reg_all=0.02, verbose=True)
algo.fit(trainset)

# Cargar datos de test
df_test = pd.read_csv("test.csv")

# Mapas para IDs originales
user_inner_id = trainset._raw2inner_id_users
item_inner_id = trainset._raw2inner_id_items

# Predecir y guardar
output_rows = []
for _, row in df_test.iterrows():
    test_id = row['ID']
    user = row['user']
    item = row['item']

    # Surprise maneja automáticamente nuevos usuarios/items como unknown
    pred = algo.predict(str(user), str(item)).est
    pred = round(max(0, min(10, pred)), 3)

    output_rows.append((test_id, pred))

# Guardar predicciones
with open("predicciones_SVD_50_20_0.02.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["ID", "rating"])
    writer.writerows(output_rows)

print("Archivo generado correctamente.")


Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Archivo generado correctamente.


In [10]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import GridSearchCV
import csv

# Cargar datos
df_train = pd.read_csv("train.csv")

# Convertir a formato surprise
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_train[['user', 'item', 'rating']], reader)

# Definir rejilla de hiperparámetros
param_grid = {
    'n_factors': [20, 50, 100],
    'n_epochs': [10, 20],
    'reg_all': [0.02, 0.05, 0.1]
}

# Búsqueda con validación cruzada
gs = GridSearchCV(SVD, param_grid, measures=['mae'], cv=3, joblib_verbose=1, n_jobs=-1)
gs.fit(data)

# Mostrar mejores hiperparámetros
print("Mejores hiperparámetros:")
print(gs.best_params['mae'])
print(f"Mejor MAE: {gs.best_score['mae']:.4f}")

# Entrenar modelo final con mejores parámetros
best_algo = gs.best_estimator['mae']
trainset = data.build_full_trainset()
best_algo.fit(trainset)

# Cargar test
df_test = pd.read_csv("test.csv")

# Predecir y guardar resultados
output_rows = []
for _, row in df_test.iterrows():
    test_id = row['ID']
    user = str(row['user'])
    item = str(row['item'])

    pred = best_algo.predict(user, item).est
    pred = round(max(0, min(10, pred)), 3)

    output_rows.append((test_id, pred))

# Guardar CSV
filename = f"predicciones_SVD_gridsearch.csv"
with open(filename, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["ID", "rating"])
    writer.writerows(output_rows)

print(f"Archivo '{filename}' generado correctamente.")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:  4.7min finished


Mejores hiperparámetros:
{'n_factors': 20, 'n_epochs': 20, 'reg_all': 0.02}
Mejor MAE: 1.2706
Archivo 'predicciones_SVD_gridsearch.csv' generado correctamente.
